In [1]:
import pandas as pd
import os

In [2]:
from utils import read_pdf, parser_model
from nlp import topic

In [4]:
data_files = [f"data/{filename}" for filename in os.listdir("data") if ".pdf" in filename]

In [9]:
text = read_pdf.path_to_sentences(data_files[1])

In [10]:
model, corpus, sentences = topic.get_topics(text, num_topics = 50)

In [11]:
def format_topics_sentences(ldamodel, corpus, texts):
    
    sent_topics_df = pd.DataFrame()

    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        row = sorted(row, key=lambda x: (x[1]), reverse=True)

        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0: 
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
                
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [12]:
df_topic_sents_keywords = format_topics_sentences(model, corpus, sentences)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,1.0,0.9272,"asset, interest, voting, shareholder, takeover...","[Proxy, Voting, Policy, Client, asset, Decembe..."
1,1,31.0,0.9211,"voting, share, right, time, double, period, is...","[underlie, principle, proxy, voting, policy, p..."
2,2,14.0,0.6815,"board, executive, Ł, company, director, struct...","[board, director]"
3,3,41.0,0.6816,"company, shareholder, voting, executive, NN, a...","[remuneration, benefit]"
4,4,29.0,0.4031,"report, annual, company, account, non, Ł, remu...","[Audit, annual, report]"
5,5,43.0,0.9475,"company, structure, ™, NN, article, IP, propos...","[capital, structure, asset, sale, M&A, special..."
6,6,9.0,0.8100,"company, NN, share, IP, ™, s, receipt, deposit...","[depositary, receipt, trust, office]"
7,7,14.0,0.8417,"board, executive, Ł, company, director, struct...","[environmental, social, governance, ESG, issue]"
8,8,41.0,0.8105,"company, shareholder, voting, executive, NN, a...","[security, lending, client, asset, 15nn, Inves..."
9,9,33.0,0.9845,"corporate, Ł, event, Governance, executive, NN...","[NN, IP, ™, s, client, include, large, small, ..."
